## Prototype of annulus analytic solution by Rhodri

In [ ]:
import numpy
from math import sqrt, atan2, cos, sin

Ra = 1.                        # Ra
R  = numpy.array([2.22, 1.22]) # Outer and inner radius:
Rr = R[::-1]                   # Inverted:
rp = R[1]+0.5                  # Radius of the anomaly - in this case, a delta function at a mid-mantle depth.
n  = 8                         # Controls "wavelength" of perturbation in tangential direction:

if n>1:
    E = -Ra*rp**(-n)/(8*(n+1)) * \
            (Rr**(2*n+2)-rp**(2*n+2))/(R[0]**(2*n+2)-R[1]**(2*n+2))
    F = -Ra*R**(2*n)*rp**(-n)/(8*(n-1)) * \
            (Rr**2*rp**(2*n)-Rr**(2*n)*rp**2)/(R[0]**2*R[1]**(2*n)-R[0]**(2*n)*R[1]**2)
    C = -F/R**(2*n-2)
    D = -E*R**(2*n+2)
else:
    raise NotImplemented()

In [ ]:
R, Rr, rp, E, F, C, D

In [ ]:
# Radial velocity:
def u_r(r, theta):
  dpsi_dtheta = n*cos(n*theta)*(C*r**n+D*r**(-n)+E*r**(n+2)+F*r**(-n+2))
  return -dpsi_dtheta/r

# Tangential velocity:
def u_theta(r, theta):
  dpsi_dr = sin(n*theta)*(C*n*r**(n-1) + D*-n*r**(-n-1) + E*(n+2)*r**(n+1) + F*(-n+2)*r**(-n+1))
  return dpsi_dr

# Function that retrieves the solution in Cartesian coordinates:
def get_cartesian_solution(X, i):
  # i==0: Above anomaly; i==1: Below anomaly (see associated .pdf)
  r     = sqrt(X[0]**2+X[1]**2)
  theta = atan2(X[1], X[0])
  ur    = u_r(r,theta)[i]
  ut    = u_theta(r,theta)[i]
  return [ur*X[0]/r - ut*X[1]/r, ur*X[1]/r + ut*X[0]/r]

In [ ]:
import underworld as uw
from underworld import function as fn
import glucifer
import math
import numpy as np

In [ ]:
from unsupported import annulus

In [ ]:
model = annulus.AnnulusConvection(elRes=(30, 300), outputPath='./output/')

In [ ]:
model.parameter_setup(Ra=1., Di=0.0, viscosity_mode=0, eqn='EB')

In [ ]:
model.checkpoint_fields['velocity'] = model.fields["velocity"]

In [ ]:
mesh = model.mesh
tField = model.fields['temperature']

In [ ]:
# element spacing h
h_r = (mesh.maxCoord[0] - mesh.minCoord[0]) / mesh.elementRes[0]

In [ ]:
h_r

In [ ]:
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

In [ ]:
r = mesh.radialData[:,0].reshape(-1,1)
t = np.pi/180.*mesh.radialData[:,1].reshape(-1,1)
tField.data[:] = gaussian(r, 1.72, 0.034)*np.cos(8*t)

In [ ]:
u_r(r[100], t[100])

In [ ]:
# radialFn = fn.math.sqrt(fn.math.dot(fn.coord(), fn.coord()))
# thetaFn = fn.math.atan2(fn.coord()[1], fn.coord()[0])

In [ ]:
stokes_system = model.system['stokes']

In [ ]:
stokes_system.fn_bodyforce = -9.8*model.fn_force

In [ ]:
fig = glucifer.Figure()
fig.append(glucifer.objects.Surface(mesh, tField, onMesh=True))

fig.append(glucifer.objects.Mesh(mesh))
fig.show()

In [ ]:
# model.initial_temperature_distribution(k=4)
# load from hdf5 file
# model.initial_temperature_distribution(h5file='annulus_output/temperature-100.h5')
# model.fields['tDot'].load('annulus_output/tDot-100.h5', interpolate=True)
# tField = model.fields['temperature']
# tDot = model.fields['tDot']
# tField.data[model.meshSets['outerNodes'].data] = 0.0
# tField.data[model.meshSets['innerNodes'].data] = 1.0
# tDot.data[model.meshSets['boundaryNodes'].data] = 0.0

In [ ]:
# model.view.show()

In [ ]:
model.simulate(maxtimesteps=1, visualiseEvery=1, checkpointEvery=1 )

In [ ]:
# if uw.utils.is_kernel():
#     vis = glucifer.lavavu.Viewer(database=model.animation.filename)
#     vis.control.Panel()
#     vis.control.ObjectList()
# #     vis.control.TimeStepper()
#     vis.control.show()